https://towardsdatascience.com/simulating-tennis-matches-with-python-or-moneyball-for-tennis-a3359df0d072

In [29]:
%pylab inline
import pandas as pd
import numpy as np
import scipy as sp
from pandas.io.json import json_normalize
import json
from datetime import date, timedelta
from time import time
from random import random

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore');

Populating the interactive namespace from numpy and matplotlib


In [20]:
#define what a big point is and
#have a separte function which can be called to get
#probability of a player winning a big point
def getBigPointProb(server):
    if server==p1:
        return p1_big_point
    elif server==p2:
        return p2_big_point
    else:
        print("Error")

In [21]:
def isBigPoint(server_points, returner_points, tiebreak):
    #server_next_point = server_points+1
    server_next_point = server_points
    #print(server_next_point)
    if tiebreak==False:
        if server_next_point >= 3 and (server_next_point - returner_points) >= 1:
            print("game point")
            return True
    else:
        if server_next_point >= 6 and abs(server_next_point - returner_points) >= 1:
            print("set point")
            return True

In [22]:
#takes a player's point string and returns tennis scoring convention
#originally, we tried to get clever with having the array extra = ['D', 'A']
#and the points%2 oscillating as index, but found that there were cases when a game went
#to multiple deuce - ads and the oscillation was returning incorrect results
#so we decided to set up if's to handle special cases when a game enters 4 points all
#in our write up, we detail how an objected oriented approach can clean up some of this logic
def getScore(pointsServer, pointsReturner, server_games, returner_games, completed_sets, tiebreaker):
    in_game = ['15', '30', '40']
    extra = ['D', 'A']
    
    display_server='0'
    display_returner='0'
    
    if tiebreaker==False:
        if pointsServer==0:
            display_server='0'
        elif pointsServer>0 and pointsServer<4:
            display_server=in_game[pointsServer-1]
        elif pointsServer>=4:
            #clean_pointsServer = pointsServer-4
            display_server = 'D'

        if pointsReturner==0:
            display_returner='0'
        elif pointsReturner>0 and pointsReturner<4:
            display_returner=in_game[pointsReturner-1]
        elif pointsReturner>=4:
            #clean_pointsReturner = pointsReturner-4
            display_returner = 'D'
        
        if (pointsServer>=4 and pointsReturner<4) or (pointsServer<4 and pointsReturner>=4):
            display_server='D'
            display_returner='D'

        if display_server=='D' and display_server=='D':
            if pointsServer>pointsReturner:
                display_server='A'
            elif pointsReturner>pointsServer:
                display_returner='A'

        if (display_server=='A' and display_returner=='A') or (display_server=='40' and display_returner=='40'):
            display_server = 'D'
            display_returner = 'D'
        if (display_server=='A' and display_returner=='40'):
            display_server = 'A'
            display_returner = 'D'
        if (display_server=='40' and display_returner=='A'):
            display_server = 'D'
            display_returner = 'A'
    else:
        display_server = str(pointsServer)
        display_returner = str(pointsReturner)
    
    if len(completed_sets)==0:
        print(display_server+"-"+display_returner+"|"+"["+str(server_games)+"-"+str(returner_games)+"]")
    else:
        completed = ""
        for sets in completed_sets:
            completed = completed+" "+str(sets[0])+":"+str(sets[1])
        print(display_server+"-"+display_returner+"|"+str(completed)+"["+str(server_games)+":"+str(returner_games)+"]")


In [41]:
#player serve simulation in a game
#after each game print out winner and
#call out if server was broken
def player_serve(server, returner, server_prob, returner_prob, gamesMatch, S, server_points_match, returner_points_match, server_games, returner_games, server_pointsGame, returner_pointsGame, completed_sets):
    if isBigPoint(server_pointsGame, returner_pointsGame, False):
        server_prob = getBigPointProb(server)
    if random() < server_prob:
        print server+" ",
        getScore(server_pointsGame, returner_pointsGame, server_games, returner_games, completed_sets, False)
        server_pointsGame += 1
        server_points_match += 1
    else:
        print server+" ",
        getScore(server_pointsGame, returner_pointsGame, server_games, returner_games, completed_sets, False)
        returner_pointsGame += 1
        returner_points_match += 1
    if max(server_pointsGame, returner_pointsGame) >= 4 and abs(server_pointsGame - returner_pointsGame) > 1:
        print "\t", server + ":", str(server_pointsGame) + ",", returner + ":", returner_pointsGame,
        if server_pointsGame > returner_pointsGame:
            server_games += 1
            print ""
        else:
            returner_games += 1
            print " -- " + returner, "broke"
        gamesMatch += 1
        return server_games, returner_games, gamesMatch, S, server_points_match, returner_points_match, server_pointsGame, returner_pointsGame

    return server_games, returner_games, gamesMatch, S, server_points_match, returner_points_match, server_pointsGame, returner_pointsGame


In [43]:
#play a set
def simulateSet(a, b, gamesMatch, S, pointsMatch1, pointsMatch2, completed_sets):
    S += 1
    gamesSet1 = 0
    gamesSet2 = 0
    while (max(gamesSet1, gamesSet2) < 6 or abs(gamesSet1 - gamesSet2) < 2) and gamesSet1 + gamesSet2 < 12: #Conditions to play another Game in this Set
        pointsGame1 = 0
        pointsGame2 = 0
        #player 1 serves
        while gamesMatch % 2 == 0:
            gamesSet1, gamesSet2, gamesMatch, S, pointsMatch1, pointsMatch2, pointsGame1, pointsGame2 = player_serve(p1, p2, a, b, gamesMatch, S, pointsMatch1, pointsMatch2, gamesSet1, gamesSet2, pointsGame1, pointsGame2, completed_sets)
        pointsGame1 = 0
        pointsGame2 = 0
        #player 2 serves, but we also incorporate in logic to end the set
        while gamesMatch % 2 == 1 and (max(gamesSet1, gamesSet2) < 6 or abs(gamesSet1 - gamesSet2) < 2) and gamesSet1 + gamesSet2 < 12:
            gamesSet2, gamesSet1, gamesMatch, S, pointsMatch2, pointsMatch1, pointsGame2, pointsGame1 = player_serve(p2, p1, b, a, gamesMatch, S, pointsMatch2, pointsMatch1, gamesSet2, gamesSet1, pointsGame2, pointsGame1, completed_sets)
    #at 6 games all we go to a tie breaker
    if gamesSet1 == 6 and gamesSet2 == 6:
        print "Set", S, "is 6-6 and going to a Tiebreaker."
    
    return gamesSet1, gamesSet2, gamesMatch, S, pointsMatch1, pointsMatch2

In [44]:
#play a tiebreak
#this function could be written much cleaner as some logic is being repeated
#however, getting the logic down first then optimizing is usually the dev process we prefer

#originally the player serving was being printed as well, but ultimately, it
#made the output results difficult to read so was commented out
def simulateTiebreaker(player1, player2, a, b, gamesMatch, pointsMatch1, pointsMatch2, completed_sets):
    pointsTie1, pointsTie2 = 0, 0           
    while max(pointsTie1, pointsTie2) < 7 or abs(pointsTie1 - pointsTie2) < 2:
        #player 1 will server first
        if gamesMatch % 2 == 0:
            while (pointsTie1 + pointsTie2) % 4 == 0 or (pointsTie1 + pointsTie2) % 4 == 3:
                server_prob = a
                if isBigPoint(pointsTie1, pointsTie2, True):
                    server_prob=getBigPointProb(player1)
                if random() < server_prob:
                    #print(player1+" ", end = "")
                    getScore(pointsTie1, pointsTie2, 6, 6, completed_sets, True)
                    pointsTie1 += 1
                    pointsMatch1 += 1
                else:
                    getScore(pointsTie1, pointsTie2, 6, 6, completed_sets, True)
                    pointsTie2 += 1
                    pointsMatch2 += 1
                if max(pointsTie1, pointsTie2) >= 7 and abs(pointsTie1 - pointsTie2) > 1:
                    print("\t", p1 + ":", str(pointsTie1) + ",", p2 + ":", pointsTie2)
                    gamesMatch += 1
                    break 
            while (max(pointsTie1, pointsTie2) < 7 or abs(pointsTie1 - pointsTie2) < 2) and ((pointsTie1 + pointsTie2) % 4 == 1 or (pointsTie1 + pointsTie2) % 4 == 2): # Conditions to continue Tiebreaker (race to 7, win by 2) and Player 2 serves (points 4N+1 and 4N+2)
                server_prob = b
                if isBigPoint(pointsTie2, pointsTie1, True):
                    server_prob=getBigPointProb(player2)
                if random() < server_prob:
                    #print(player2+" ", end = "")
                    getScore(pointsTie1, pointsTie2, 6, 6, completed_sets, True)
                    pointsTie2 += 1
                    pointsMatch2 += 1
                else:
                    getScore(pointsTie1, pointsTie2, 6, 6, completed_sets, True)
                    pointsTie1 += 1
                    pointsMatch1 += 1
                if max(pointsTie1, pointsTie2) >= 7 and abs(pointsTie1 - pointsTie2) > 1:
                    print "\t", p1 + ":", str(pointsTie1) + ",", p2 + ":", pointsTie2
                    break
        
        #player 2 will server first
        if gamesMatch % 2 == 1:
            while (pointsTie1 + pointsTie2) % 4 == 1 or (pointsTie1 + pointsTie2) % 4 == 2:
                server_prob =  a
                if isBigPoint(pointsTie1, pointsTie2, True):
                    server_prob=getBigPointProb(player1)
                if random() < server_prob:
                    #print(player1+" ", end = "")
                    getScore(pointsTie1, pointsTie2, 6, 6, completed_sets, True)
                    pointsTie1 += 1
                    pointsMatch1 += 1
                else:
                    getScore(pointsTie1, pointsTie2, 6, 6, completed_sets, True)
                    pointsTie2 += 1
                    pointsMatch2 += 1
                if max(pointsTie1, pointsTie2) >= 7 and abs(pointsTie1 - pointsTie2) > 1:
                    print "\t", p1 + ":", str(pointsTie1) + ",", p2 + ":", pointsTie2
                    break 
            while (max(pointsTie2, pointsTie1) < 7 or abs(pointsTie1 - pointsTie2) < 2) and ((pointsTie1 + pointsTie2) % 4 == 0 or (pointsTie1 + pointsTie2) % 4 == 3): # Conditions to continue Tiebreaker (race to 7, win by 2) and Player 2 serves (points 4N and 4N+3)
                server_prob =  b
                if isBigPoint(pointsTie2, pointsTie1, True):
                    server_prob=getBigPointProb(player2)
                if random() < server_prob:
                    #print(player2+" ", end = "")
                    getScore(pointsTie1, pointsTie2, 6, 6, completed_sets, True)
                    pointsTie2 += 1
                    pointsMatch2 += 1
                else:
                    getScore(pointsTie1, pointsTie2, 6, 6, completed_sets, True)
                    pointsTie1 += 1
                    pointsMatch1 += 1
                if max(pointsTie1, pointsTie2) >= 7 and abs(pointsTie1 - pointsTie2) > 1:
                    print "\t", p1 + ":", str(pointsTie1) + ",", p2 + ":", pointsTie2
                    break                             
    gamesMatch += 1
    return pointsTie1, pointsTie2, gamesMatch, pointsMatch1, pointsMatch2


In [45]:
#displays the running score in the way an umpire would announce the score at
#the end of a set
def printSetMatchSummary(p1, p2, gamesSet1, gamesSet2, S, pointsTie1, pointsTie2, setsMatch1, setsMatch2):
    if gamesSet1 > gamesSet2:
        setsMatch1 += 1
        print p1.upper(), "wins Set", str(S) + ":", gamesSet1, "games to", str(gamesSet2) + "."
    elif gamesSet2 > gamesSet1:
        setsMatch2 += 1
        print p2.upper(), "wins Set", str(S) + ":", gamesSet2, "games to", str(gamesSet1) + "." 
    elif gamesSet1 == gamesSet2:
        if pointsTie1 > pointsTie2:
            setsMatch1 += 1
            print p1.upper(), "wins Set", str(S) + ": 7 games to 6 (" + str(pointsTie1) + "-" + str(pointsTie2) + ")."
        else:
            setsMatch2 += 1
            print p2.upper(), "wins Set", str(S) + ": 7 games to 6 (" + str(pointsTie2) + "-" + str(pointsTie1) + ")."
    print "After", S, "Sets:", p1, str(setsMatch1) + ",", p2, str(setsMatch2) + ".\n"
    return setsMatch1, setsMatch2


In [46]:
#prints out final winner and number of sets won
def pointsMatchSummary(p1, p2, setsMatch1, setsMatch2, pointsMatch1, pointsMatch2):
        if setsMatch1 == 3:
            print p1.upper(), "(" + str(a) + ")", "beat", p2, "(" + str(b) + ") by", setsMatch1, "Sets to", str(setsMatch2) + "."
            winner.append(p1)
        else:
            print p2.upper(), "(" + str(b) + ")", "beat", p1, "(" + str(a) + ") by", setsMatch2, "Sets to", str(setsMatch1) + "."
            winner.append(p2)


In [48]:
#this control flow a single simulation
winner = []
completed_sets = []
S = 0
gamesMatch = 0

#in all subscripted variables
#the subscript refers to the player
#for example, setsMatch1 is sets won by player1 and
#setsMatch2 is sets won by player2
pointsMatch1, pointsMatch2 = 0, 0
setsMatch1, setsMatch2 = 0, 0
pointsTie1, pointsTie2 = 0, 0
pointsGame1, pointsGame2 = 0, 0

#initialize player one and two
#a is ps1 and b is ps2
#p1_big_point and p2_big_point are the probability
#of p1 and p2 winning on a big point, respectively
p1 = "A"
p2 = "B"
a = 0.64
b = 0.62
p1_big_point = 0.70
p2_big_point = 0.68

while S < 5 and max(setsMatch1, setsMatch2) < 3:
    gamesSet1, gamesSet2, gamesMatch, S, pointsMatch1, pointsMatch2 = simulateSet(a, b, gamesMatch, S, 
                                                                                  pointsMatch1, pointsMatch2, 
                                                                                  completed_sets)
    print ""
    if gamesSet1 == 6 and gamesSet2 == 6:
        pointsTie1, pointsTie2, gamesMatch, pointsMatch1, pointsMatch2 = simulateTiebreaker(p1, p2, a, b, 
                                                                                            gamesMatch, pointsMatch1, 
                                                                                            pointsMatch2, 
                                                                                            completed_sets)
    
    setsMatch1, setsMatch2 = printSetMatchSummary(p1, p2, gamesSet1, gamesSet2, 
                                                  S, pointsTie1, pointsTie2, 
                                                  setsMatch1, setsMatch2)
    
    if gamesSet1 == 6 and gamesSet2 == 6:
        if pointsTie1 > pointsTie2:
            completed_sets.append([gamesSet1+1, gamesSet2])
        else:
            completed_sets.append([gamesSet1, gamesSet2+1])
    else:
        completed_sets.append([gamesSet1, gamesSet2])

pointsMatchSummary(p1, p2, setsMatch1, setsMatch2, pointsMatch1, pointsMatch2)

A  0-0|[0-0]
A  15-0|[0-0]
A  15-15|[0-0]
A  30-15|[0-0]
A  30-30|[0-0]
game point
A  40-30|[0-0]
	A: 4, B: 2 
B  0-0|[0-1]
B  0-15|[0-1]
B  0-30|[0-1]
B  15-30|[0-1]
B  30-30|[0-1]
game point
B  40-30|[0-1]
	B: 4, A: 2 
A  0-0|[1-1]
A  0-15|[1-1]
A  15-15|[1-1]
A  30-15|[1-1]
A  30-30|[1-1]
game point
A  40-30|[1-1]
	A: 4, B: 2 
B  0-0|[1-2]
B  15-0|[1-2]
B  15-15|[1-2]
B  30-15|[1-2]
B  30-30|[1-2]
game point
B  40-30|[1-2]
	B: 4, A: 2 
A  0-0|[2-2]
A  15-0|[2-2]
A  15-15|[2-2]
A  30-15|[2-2]
A  30-30|[2-2]
game point
A  40-30|[2-2]
	A: 4, B: 2 
B  0-0|[2-3]
B  15-0|[2-3]
B  30-0|[2-3]
B  30-15|[2-3]
B  30-30|[2-3]
game point
B  40-30|[2-3]
B  D-D|[2-3]
game point
B  A-D|[2-3]
	B: 5, A: 3 
A  0-0|[3-3]
A  15-0|[3-3]
A  30-0|[3-3]
A  30-15|[3-3]
A  30-30|[3-3]
A  30-40|[3-3]
A  D-D|[3-3]
game point
A  A-D|[3-3]
	A: 5, B: 3 
B  0-0|[3-4]
B  0-15|[3-4]
B  15-15|[3-4]
B  15-30|[3-4]
B  15-40|[3-4]
	B: 1, A: 4  -- A broke
A  0-0|[5-3]
A  15-0|[5-3]
A  30-0|[5-3]
game point
A  40-0|[5-3]
	

A  0-0| 6:3 6:2 6:7 6:7[4:5]
A  15-0| 6:3 6:2 6:7 6:7[4:5]
A  15-15| 6:3 6:2 6:7 6:7[4:5]
A  30-15| 6:3 6:2 6:7 6:7[4:5]
game point
A  40-15| 6:3 6:2 6:7 6:7[4:5]
	A: 4, B: 1 
B  0-0| 6:3 6:2 6:7 6:7[5:5]
B  15-0| 6:3 6:2 6:7 6:7[5:5]
B  30-0| 6:3 6:2 6:7 6:7[5:5]
game point
B  40-0| 6:3 6:2 6:7 6:7[5:5]
	B: 4, A: 0 
A  0-0| 6:3 6:2 6:7 6:7[5:6]
A  15-0| 6:3 6:2 6:7 6:7[5:6]
A  30-0| 6:3 6:2 6:7 6:7[5:6]
game point
A  40-0| 6:3 6:2 6:7 6:7[5:6]
	A: 4, B: 0 
Set 5 is 6-6 and going to a Tiebreaker.

0-0| 6:3 6:2 6:7 6:7[6:6]
1-0| 6:3 6:2 6:7 6:7[6:6]
1-1| 6:3 6:2 6:7 6:7[6:6]
2-1| 6:3 6:2 6:7 6:7[6:6]
3-1| 6:3 6:2 6:7 6:7[6:6]
4-1| 6:3 6:2 6:7 6:7[6:6]
4-2| 6:3 6:2 6:7 6:7[6:6]
5-2| 6:3 6:2 6:7 6:7[6:6]
6-2| 6:3 6:2 6:7 6:7[6:6]
	A: 7, B: 2
A wins Set 5: 7 games to 6 (7-2).
After 5 Sets: A 3, B 2.

A (0.64) beat B (0.62) by 3 Sets to 2.


In [54]:
#this control flow module runs 10 simulations
#and stores the winner of each simulation in winner=[]
winner = []
p1 = "A"
p2 = "B"
a = 0.64
b = 0.62
p1_big_point = 0.70
p2_big_point = 0.68
#run 1000 runs of the simulation
for ii in range(0, 10):
    completed_sets = []
    S = 0
    gamesMatch = 0
    pointsMatch1, pointsMatch2 = 0, 0
    setsMatch1, setsMatch2 = 0, 0
    pointsTie1, pointsTie2 = 0, 0
    pointsGame1, pointsGame2 = 0, 0
    
    while S < 5 and max(setsMatch1, setsMatch2) < 3:
        gamesSet1, gamesSet2, gamesMatch, S, pointsMatch1, pointsMatch2 = simulateSet(a, b, gamesMatch, S, 
                                                                                      pointsMatch1, pointsMatch2, completed_sets)
        print()
        if gamesSet1 == 6 and gamesSet2 == 6:
            pointsTie1, pointsTie2, gamesMatch, pointsMatch1, pointsMatch2 = simulateTiebreaker(p1, p2, a, b, gamesMatch, 
                                                                                                pointsMatch1, pointsMatch2, completed_sets)
setsMatch1, setsMatch2 = printSetMatchSummary(p1, p2, gamesSet1, gamesSet2, S, 
                                                      pointsTie1, pointsTie2, setsMatch1, setsMatch2)
if gamesSet1 == 6 and gamesSet2 == 6:
            if pointsTie1 > pointsTie2:
                completed_sets.append([gamesSet1+1, gamesSet2])
            else:
                completed_sets.append([gamesSet1, gamesSet2+1])
else:
            completed_sets.append([gamesSet1, gamesSet2])
pointsMatchSummary(p1, p2, setsMatch1, setsMatch2, pointsMatch1, pointsMatch2)


A  0-0|[0-0]
A  15-0|[0-0]
A  30-0|[0-0]
A  30-15|[0-0]
game point
A  40-15|[0-0]
	A: 4, B: 1 
B  0-0|[0-1]
B  15-0|[0-1]
B  30-0|[0-1]
game point
B  40-0|[0-1]
	B: 4, A: 0 
A  0-0|[1-1]
A  0-15|[1-1]
A  15-15|[1-1]
A  30-15|[1-1]
game point
A  40-15|[1-1]
game point
A  40-30|[1-1]
A  D-D|[1-1]
A  D-A|[1-1]
	A: 3, B: 5  -- B broke
B  0-0|[2-1]
B  0-15|[2-1]
B  15-15|[2-1]
B  15-30|[2-1]
B  30-30|[2-1]
game point
B  40-30|[2-1]
	B: 4, A: 2 
A  0-0|[1-3]
A  15-0|[1-3]
A  15-15|[1-3]
A  30-15|[1-3]
game point
A  40-15|[1-3]
game point
A  40-30|[1-3]
A  D-D|[1-3]
A  D-A|[1-3]
A  D-D|[1-3]
A  D-A|[1-3]
	A: 4, B: 6  -- B broke
B  0-0|[4-1]
B  15-0|[4-1]
B  30-0|[4-1]
game point
B  40-0|[4-1]
	B: 4, A: 0 
A  0-0|[1-5]
A  15-0|[1-5]
A  30-0|[1-5]
game point
A  40-0|[1-5]
	A: 4, B: 0 
B  0-0|[5-2]
B  0-15|[5-2]
B  15-15|[5-2]
B  30-15|[5-2]
game point
B  40-15|[5-2]
	B: 4, A: 1 
()
A  0-0|[0-0]
A  0-15|[0-0]
A  15-15|[0-0]
A  30-15|[0-0]
A  30-30|[0-0]
game point
A  40-30|[0-0]
	A: 4, B: 2 
B  

game point
B  40-0|[3-4]
	B: 4, A: 0 
A  0-0|[4-4]
A  15-0|[4-4]
A  30-0|[4-4]
A  30-15|[4-4]
game point
A  40-15|[4-4]
	A: 4, B: 1 
B  0-0|[4-5]
B  15-0|[4-5]
B  15-15|[4-5]
B  30-15|[4-5]
game point
B  40-15|[4-5]
game point
B  40-30|[4-5]
B  D-D|[4-5]
game point
B  A-D|[4-5]
	B: 5, A: 3 
A  0-0|[5-5]
A  0-15|[5-5]
A  15-15|[5-5]
A  30-15|[5-5]
game point
A  40-15|[5-5]
game point
A  40-30|[5-5]
	A: 4, B: 2 
B  0-0|[5-6]
B  15-0|[5-6]
B  30-0|[5-6]
game point
B  40-0|[5-6]
	B: 4, A: 0 
Set 5 is 6-6 and going to a Tiebreaker.
()
0-0|[6-6]
1-0|[6-6]
2-0|[6-6]
3-0|[6-6]
4-0|[6-6]
4-1|[6-6]
4-2|[6-6]
4-3|[6-6]
5-3|[6-6]
5-4|[6-6]
6-4|[6-6]
	A: 7, B: 4
A  0-0|[0-0]
A  0-15|[0-0]
A  0-30|[0-0]
A  15-30|[0-0]
A  30-30|[0-0]
A  30-40|[0-0]
A  D-D|[0-0]
game point
A  A-D|[0-0]
	A: 5, B: 3 
B  0-0|[0-1]
B  15-0|[0-1]
B  30-0|[0-1]
game point
B  40-0|[0-1]
	B: 4, A: 0 
A  0-0|[1-1]
A  15-0|[1-1]
A  30-0|[1-1]
game point
A  40-0|[1-1]
	A: 4, B: 0 
B  0-0|[1-2]
B  15-0|[1-2]
B  30-0|[1-2]
B  30-1

B  30-30|[0-1]
game point
B  40-30|[0-1]
	B: 4, A: 2 
A  0-0|[1-1]
A  15-0|[1-1]
A  15-15|[1-1]
A  15-30|[1-1]
A  15-40|[1-1]
A  30-40|[1-1]
	A: 2, B: 4  -- B broke
B  0-0|[2-1]
B  15-0|[2-1]
B  30-0|[2-1]
game point
B  40-0|[2-1]
	B: 4, A: 0 
A  0-0|[1-3]
A  15-0|[1-3]
A  30-0|[1-3]
game point
A  40-0|[1-3]
game point
A  40-15|[1-3]
	A: 4, B: 1 
B  0-0|[3-2]
B  0-15|[3-2]
B  15-15|[3-2]
B  15-30|[3-2]
B  30-30|[3-2]
game point
B  40-30|[3-2]
	B: 4, A: 2 
A  0-0|[2-4]
A  15-0|[2-4]
A  30-0|[2-4]
game point
A  40-0|[2-4]
game point
A  40-15|[2-4]
	A: 4, B: 1 
B  0-0|[4-3]
B  15-0|[4-3]
B  30-0|[4-3]
B  30-15|[4-3]
B  30-30|[4-3]
B  30-40|[4-3]
	B: 2, A: 4  -- A broke
A  0-0|[4-4]
A  15-0|[4-4]
A  30-0|[4-4]
A  30-15|[4-4]
game point
A  40-15|[4-4]
	A: 4, B: 1 
B  0-0|[4-5]
B  15-0|[4-5]
B  30-0|[4-5]
B  30-15|[4-5]
B  30-30|[4-5]
B  30-40|[4-5]
B  D-D|[4-5]
B  D-A|[4-5]
B  D-D|[4-5]
B  D-A|[4-5]
B  D-D|[4-5]
game point
B  A-D|[4-5]
	B: 7, A: 5 
A  0-0|[5-5]
A  0-15|[5-5]
A  0-30|[5-5]
A

game point
B  40-15|[1-0]
	B: 4, A: 1 
A  0-0|[0-2]
A  15-0|[0-2]
A  30-0|[0-2]
A  30-15|[0-2]
game point
A  40-15|[0-2]
	A: 4, B: 1 
B  0-0|[2-1]
B  15-0|[2-1]
B  30-0|[2-1]
game point
B  40-0|[2-1]
	B: 4, A: 0 
A  0-0|[1-3]
A  0-15|[1-3]
A  0-30|[1-3]
A  0-40|[1-3]
A  15-40|[1-3]
A  30-40|[1-3]
A  D-D|[1-3]
game point
A  A-D|[1-3]
	A: 5, B: 3 
B  0-0|[3-2]
B  0-15|[3-2]
B  0-30|[3-2]
B  15-30|[3-2]
B  30-30|[3-2]
game point
B  40-30|[3-2]
	B: 4, A: 2 
A  0-0|[2-4]
A  15-0|[2-4]
A  30-0|[2-4]
A  30-15|[2-4]
game point
A  40-15|[2-4]
game point
A  40-30|[2-4]
	A: 4, B: 2 
B  0-0|[4-3]
B  15-0|[4-3]
B  30-0|[4-3]
game point
B  40-0|[4-3]
	B: 4, A: 0 
A  0-0|[3-5]
A  0-15|[3-5]
A  0-30|[3-5]
A  15-30|[3-5]
A  15-40|[3-5]
A  30-40|[3-5]
A  D-D|[3-5]
A  D-A|[3-5]
A  D-D|[3-5]
A  D-A|[3-5]
	A: 4, B: 6  -- B broke
()
B  0-0|[0-0]
B  15-0|[0-0]
B  15-15|[0-0]
B  30-15|[0-0]
B  30-30|[0-0]
game point
B  40-30|[0-0]
	B: 4, A: 2 
A  0-0|[0-1]
A  15-0|[0-1]
A  15-15|[0-1]
A  30-15|[0-1]
game poin

	A: 4, B: 1 
B  0-0|[4-5]
B  0-15|[4-5]
B  0-30|[4-5]
B  0-40|[4-5]
B  15-40|[4-5]
	B: 1, A: 4  -- A broke
()
A  0-0|[0-0]
A  0-15|[0-0]
A  15-15|[0-0]
A  15-30|[0-0]
A  15-40|[0-0]
	A: 1, B: 4  -- B broke
B  0-0|[1-0]
B  15-0|[1-0]
B  30-0|[1-0]
game point
B  40-0|[1-0]
game point
B  40-15|[1-0]
	B: 4, A: 1 
A  0-0|[0-2]
A  15-0|[0-2]
A  15-15|[0-2]
A  15-30|[0-2]
A  15-40|[0-2]
	A: 1, B: 4  -- B broke
B  0-0|[3-0]
B  15-0|[3-0]
B  30-0|[3-0]
B  30-15|[3-0]
B  30-30|[3-0]
B  30-40|[3-0]
B  D-D|[3-0]
game point
B  A-D|[3-0]
	B: 5, A: 3 
A  0-0|[0-4]
A  15-0|[0-4]
A  30-0|[0-4]
game point
A  40-0|[0-4]
	A: 4, B: 0 
B  0-0|[4-1]
B  15-0|[4-1]
B  30-0|[4-1]
game point
B  40-0|[4-1]
	B: 4, A: 0 
A  0-0|[1-5]
A  15-0|[1-5]
A  15-15|[1-5]
A  30-15|[1-5]
game point
A  40-15|[1-5]
	A: 4, B: 1 
B  0-0|[5-2]
B  0-15|[5-2]
B  15-15|[5-2]
B  30-15|[5-2]
game point
B  40-15|[5-2]
game point
B  40-30|[5-2]
	B: 4, A: 2 
()
A  0-0|[0-0]
A  15-0|[0-0]
A  30-0|[0-0]
game point
A  40-0|[0-0]
	A: 4, B: 0 

A  40-0|[5-5]
	A: 4, B: 0 
B  0-0|[5-6]
B  0-15|[5-6]
B  15-15|[5-6]
B  30-15|[5-6]
game point
B  40-15|[5-6]
game point
B  40-30|[5-6]
B  D-D|[5-6]
game point
B  A-D|[5-6]
	B: 5, A: 3 
Set 4 is 6-6 and going to a Tiebreaker.
()
0-0|[6-6]
1-0|[6-6]
1-1|[6-6]
2-1|[6-6]
3-1|[6-6]
3-2|[6-6]
4-2|[6-6]
4-3|[6-6]
5-3|[6-6]
6-3|[6-6]
6-4|[6-6]
set point
6-5|[6-6]
6-6|[6-6]
set point
7-6|[6-6]
7-7|[6-6]
set point
7-8|[6-6]
8-8|[6-6]
set point
8-9|[6-6]
	A: 8, B: 10
B  0-0|[0-0]
B  15-0|[0-0]
B  30-0|[0-0]
B  30-15|[0-0]
game point
B  40-15|[0-0]
game point
B  40-30|[0-0]
B  D-D|[0-0]
B  D-A|[0-0]
B  D-D|[0-0]
B  D-A|[0-0]
B  D-D|[0-0]
game point
B  A-D|[0-0]
	B: 7, A: 5 
A  0-0|[0-1]
A  15-0|[0-1]
A  15-15|[0-1]
A  15-30|[0-1]
A  30-30|[0-1]
game point
A  40-30|[0-1]
	A: 4, B: 2 
B  0-0|[1-1]
B  0-15|[1-1]
B  15-15|[1-1]
B  15-30|[1-1]
B  30-30|[1-1]
B  30-40|[1-1]
B  D-D|[1-1]
B  D-A|[1-1]
B  D-D|[1-1]
B  D-A|[1-1]
B  D-D|[1-1]
B  D-A|[1-1]
B  D-D|[1-1]
B  D-A|[1-1]
B  D-D|[1-1]
game point
B 

	B: 4, A: 2 
()
B wins Set 5: 6 games to 4.
After 5 Sets: A 0, B 1.

B (0.62) beat A (0.64) by 1 Sets to 0.
